In [1]:
import torch

from torchvision.transforms import ToTensor, Compose, Normalize

from data.dataset import get_data

from model.model import YOLONet
from model.train import train
from model.loss import DetectionLoss

from eval.display import plot_losses

DATASET = "../datasets/Human-Fall-Detection"
BATCH_SIZE = 8

SUBSET = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
from data.dataset import transform_data

data = get_data(DATASET, transform_data, device=device, batch_size=BATCH_SIZE, end=BATCH_SIZE if SUBSET else None)
 
# train_loader, val_loader, test_loader = list(data["dataloaders"].values())

In [3]:
with torch.device(device):
  model = YOLONet(3, ch=(64,32,16))
  model.load_state_dict(torch.load('model_softmax_v3_last.pt'))
  criterion = DetectionLoss(model.head)

# res = train(model, train_loader, val_loader, test_loader, 2, criterion)


In [4]:
# torch.save(model.state_dict(), 'model_sigmoid_e14_last.pt')

In [5]:
_, _, test_loader = list(data["dataloaders"].values())

model.eval()
for image, label in test_loader:
  break
print(image.shape)
out = model(image)


c:\Users\yapzh\CodeProjects\dl-fall-detection\dl-fall-detection\data\dataset.py:98: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  image = torch.tensor(image, dtype=torch.float32).to(self.device).permute(2, 0, 1)


torch.Size([8, 3, 640, 640])


c:\Users\yapzh\miniconda3\envs\dlenv\Lib\site-packages\torch\functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:3550.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [6]:
out

[tensor([[  1.0000,   0.9999,  24.2878,  35.0611,  89.2133, 209.9072],
         [  1.0000,   0.9997,  80.0276,  19.0029, 112.2757, 203.1516],
         [  1.0000,   0.9993, 171.5294,  30.8013,  92.5540, 162.1658],
         ...,
         [  0.0000,   1.0000,  95.2032, 578.8314, 216.2743, 461.7449],
         [  1.0000,   1.0000, 448.3850, 608.9814, 500.9737, 528.9914],
         [  1.0000,   1.0000, 679.0433, 595.4932, 487.2999, 556.9459]],
        device='cuda:0', grad_fn=<CatBackward0>),
 tensor([[  1.0000,   0.9998, 119.8007,  41.7199, 106.8405, 185.3803],
         [  1.0000,   0.9991, 170.3075,  57.0745, 117.3785, 194.1051],
         [  1.0000,   0.9992, 214.3450,  64.2392, 123.2739, 207.6713],
         [  1.0000,   1.0000, 268.2493,  67.7991, 119.4990, 213.1524],
         [  1.0000,   1.0000, 316.2565,  60.6088, 120.6039, 211.7044],
         [  1.0000,   0.9999, 364.5763,  58.0792, 114.4532, 200.3321],
         [  1.0000,   0.9999, 408.9846,  63.8830, 115.3589, 177.6532],
         [  

In [89]:
from utils import dist2bbox, make_anchors, bbox_iou

def nms(boxes, scores, iou_threshold=0.5):
    """Sort confidence scores of predicted bounding boxes."""
    B = torch.argsort(scores, dim=-1, descending=True)
    keep = []  # Indices of predicted bounding boxes that will be kept
    while B.numel() > 0:
        i = B[0]
        keep.append(i)
        if B.numel() == 1: break
        iou = bbox_iou(boxes[:, i], boxes[:, B[1:]].reshape(-1))
        inds = torch.nonzero(iou <= iou_threshold).reshape(-1)
        B = B[inds + 1]
    return torch.tensor(keep, device=boxes.device)



def multibox_detection(cls_probs, pred_bboxes, nms_threshold=0.5,
                       pos_threshold=0.009999999):
    """Predict bounding boxes using non-maximum suppression."""
    device, batch_size = cls_probs.device, cls_probs.shape[0]
    num_classes, num_anchors = cls_probs.shape[1], cls_probs.shape[2]

    cls_probs = cls_probs.exp()

    out = []
    for i in range(batch_size):
        cls_prob, predicted_bb = cls_probs[i], pred_bboxes[i]
        conf, class_id = torch.max(cls_prob[1:], 0)
        keep = nms(predicted_bb, conf, nms_threshold)
        # Find all non-`keep` indices and set the class to background
        all_idx = torch.arange(num_anchors, dtype=torch.long, device=device)
        combined = torch.cat((keep, all_idx))
        uniques, counts = combined.unique(return_counts=True)
        non_keep = uniques[counts == 1]
        # all_id_sorted = torch.cat((keep, non_keep))
        class_id[non_keep] = -1
        # class_id = class_id[all_id_sorted]
        # return all_id_sorted
        # conf, predicted_bb = conf[all_id_sorted], predicted_bb[:, all_id_sorted]
        # Here `pos_threshold` is a threshold for positive (non-background)
        # predictions
        # below_min_idx = (conf < pos_threshold)
        # class_id[below_min_idx] = -1
        # conf[below_min_idx] = 1 - conf[below_min_idx]
        # pred_info = torch.cat((class_id.unsqueeze(1),
        #                        conf.unsqueeze(1),
        #                        predicted_bb.permute(1,0),
        #                        ), dim=1)

        fg_mask = (conf >= pos_threshold) & (counts != 1)
        pred_info = torch.cat((class_id[fg_mask].unsqueeze(1),
                               conf[fg_mask].unsqueeze(1),
                               predicted_bb.permute(1,0)[fg_mask],
                               ), dim=1)
        out.append(pred_info)
    return out


In [23]:
classes[0].shape, dbox[0].shape

(torch.Size([3, 2100]), torch.Size([4, 2100]))

In [98]:
classes.exp()[1, :, :10].permute(1, 0)

tensor([[9.9911e-09, 4.3336e-05, 9.9996e-01],
        [9.8091e-09, 3.1718e-05, 9.9997e-01],
        [9.0353e-09, 2.6302e-05, 9.9997e-01],
        [7.5634e-09, 2.7108e-05, 9.9997e-01],
        [7.8381e-09, 2.5418e-05, 9.9997e-01],
        [8.1069e-09, 2.5375e-05, 9.9997e-01],
        [8.6025e-09, 2.7418e-05, 9.9997e-01],
        [1.1589e-08, 3.4011e-05, 9.9997e-01],
        [1.9068e-08, 4.2501e-05, 9.9996e-01],
        [4.8407e-08, 1.2666e-04, 9.9987e-01]], device='cuda:0',
       grad_fn=<PermuteBackward0>)

In [90]:
cls_prob, predicted_bb = classes[0], dbox[0]
nms_threshold = 0.0009

# print(cls_prob.shape, predicted_bb.shape)
# conf, class_id = torch.max(cls_prob, 0)
# keep = nms(predicted_bb, conf.exp(), nms_threshold)

out = multibox_detection(classes, dbox, nms_threshold)

In [73]:
out.shape

torch.Size([8, 2100, 6])

In [91]:
out[0]

tensor([[  1.0000,   1.0000, 510.1026, 136.5169,  96.0867, 346.4762],
        [  1.0000,   1.0000, 615.4004, 166.5301,  56.2640, 289.9114],
        [  1.0000,   1.0000, 293.2651, 121.2922, 120.0957, 332.7563],
        [  1.0000,   1.0000,  18.9957, 567.5600,  42.8834, 208.7926],
        [  1.0000,   1.0000, 138.0385, 649.0853, 183.9688, 221.9958],
        [  0.0000,   1.0000, 111.3201, 231.0661, 220.1606, 462.0365],
        [  1.0000,   1.0000, 516.7999, 480.5151, 484.1750, 341.5541]],
       device='cuda:0', grad_fn=<CatBackward0>)

In [7]:
for o in out:
    print(o.shape)

torch.Size([183, 6])
torch.Size([163, 6])
torch.Size([167, 6])
torch.Size([164, 6])
torch.Size([181, 6])
torch.Size([173, 6])
torch.Size([187, 6])
torch.Size([175, 6])


In [22]:
classes[0, :, 0].exp()

tensor([1.1238e-08, 5.6009e-05, 9.9994e-01], device='cuda:0',
       grad_fn=<ExpBackward0>)

In [21]:
classes.argmax(1).min()

tensor(0, device='cuda:0')